In [8]:
import os, sys
import pandas as pd
import pickle
from os.path import dirname, abspath
d = os.path.abspath(os.path.join('..'))
sys.path.append(d)
from sklearn.metrics import mean_absolute_error
from sklearn import tree
import seaborn as sns
import matplotlib
matplotlib.rcParams.update({'font.size': 8, 'figure.figsize' : [1.5*3.125, 1.5*1.5],
                           'legend.fontsize': 10, 'legend.fancybox': True,
                           'font.family': 'serif', 'font.sans-serif': 'Times',
                            'xtick.major.width': 0.2, 'xtick.minor.width': 2*0.2,
                            'ytick.major.width': 0.2, 'ytick.minor.width': 2*0.2,
                            'text.usetex': False, 'axes.labelsize': 7*2
                           })
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import matplotlib.pyplot as plt
import numpy as np
import re
from glob import glob
import zipfile
from io import TextIOWrapper



In [9]:
%run "../../vca_perf/util/webrtc_reader.py"

# Extraction Logic

In [3]:
project_config = {"is_string": 1, "webrtc_format": 2}
def check_log(input_zip):
    for name in input_zip.namelist():
        if "log" not in name:
            continue
        lines = TextIOWrapper(input_zip.open(name))
        for line in lines:
            if "Webrtc content" in line:
                substr = line[line.find("{"): -1].replace("'", '"')
                json_obj = json.loads(substr)
                if "call_duration" not in json_obj:
                    return 1
                else:
                    return 0

def get_mean_qoe(input_zip):                  
    for name in input_zip.namelist():
        if "json" not in name:
            continue
        webrtc_reader = WebRTCReader(name, dataset="netrics_data")
        df_webrtc = webrtc_reader.get_webrtc()
        #print(df_webrtc.columns)
        if df_webrtc.empty:
            return -1
        return df_webrtc["framesReceivedPerSecond"].mean(), df_webrtc["bitrate"].mean(), df_webrtc["frameHeight"].mean(), df_webrtc["frame_jitter"].mean()


In [4]:
data_dir = "/scratch/datasets/broadband/dev1/nm-exp-active-netrics/data/s3/sync/netrics_mngd/nm-exp-active-netrics/beta/"
devices = [x for x in glob(f"{data_dir}/*")]#["nm-mngd-20210518-f9511f7a", "nm-mngd-20210518-f2242dfd", "nm-mngd-20210518-c22d980b"]
device_ids = [os.path.basename(x) for x in glob(f"{data_dir}/*")]
for i in range(len(devices)):
    if not os.path.exists(f"/scratch/datasets/vcqoe/13May/{device_ids[i]}/"):
        os.mkdir(f"/scratch/datasets/vcqoe/13May/{device_ids[i]}/")

In [5]:
dates = ['20230513', '20230514', '20230515', '20230516']

In [6]:
# data_dir = "/home/taveesh/vca-qoe-inference/data/netrics_data"
log = {}
for device in device_ids:
    log[device] = {"total": 0, "corrupt": 0, "failed": 0}
    for d in dates:
        files = glob(f'{data_dir}/{device}/{d}*/zip/*vca*.zip')
        for file in files:
            os.system(f"cp {file} /scratch/datasets/vcqoe/13May/{device}/")
            log[device]["total"] += 1
            try:
                input_zip= zipfile.ZipFile(file)
            except:
                log[device]["corrupt"] += 1
                continue
            log[device]["failed"] += check_log(input_zip)
            zipfile.ZipFile(file).extractall(f"/scratch/datasets/vcqoe/13May/{device}/extract")
#         print(get_mean_qoe(input_zip))
        #extractiondir = f'{path}/extract'

#zip = zipfile.ZipFile(zipfilepath)
#zip.extractall(path=extractiondir)

In [7]:
s = 0
for x in log:
    print(x)
    if log[x]['total'] > 0:
        print(log[x])
        s += (log[x]['total'] - log[x]['failed'] - log[x]['corrupt'])
print(s)

nm-mngd-20220914-164dbe45
nm-mngd-20210518-075ab2f0
nm-mngd-20210518-6df15aaf
{'total': 145, 'corrupt': 0, 'failed': 63}
nm-mngd-20210518-28abe768
{'total': 89, 'corrupt': 0, 'failed': 36}
nm-mngd-20210518-0ca06c37
nm-mngd-20220628-07c02d28
nm-mngd-20221025-e7ab17cc
{'total': 47, 'corrupt': 0, 'failed': 10}
nm-mngd-20230306-abd14991
nm-mngd-20210518-005bb669
{'total': 103, 'corrupt': 0, 'failed': 25}
nm-mngd-20230202-e731ce2c
nm-mngd-20221108-938caf46
nm-mngd-20230306-4d5274ae
nm-mngd-20210621-2d29599c
nm-mngd-20210518-221fcce5
nm-mngd-20230306-e232fab7
nm-mngd-20230306-0f628bf2
nm-mngd-20210518-f2242dfd
{'total': 99, 'corrupt': 0, 'failed': 28}
nm-mngd-20210518-19941862
nm-mngd-20210518-f9511f7a
{'total': 59, 'corrupt': 0, 'failed': 16}
nm-mngd-20221017-a79b4ff5
nm-mngd-20210519-84594993
{'total': 59, 'corrupt': 0, 'failed': 8}
nm-mngd-20210518-c22d980b
{'total': 48, 'corrupt': 0, 'failed': 3}
README
nm-mngd-20220914-9d4c196c
nm-mngd-20210511-9d9824d4
{'total': 144, 'corrupt': 0, 'fai